In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd

%load_ext lab_black
%load_ext watermark

# Gesell

This example demonstrates ...

Adapted from [unit 9: Gesell.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/Gesell.odc).

## Associated lecture video: Unit 9 Lesson 6

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=93" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement

Gesell Developmental Schedules  measure  child development. The scoring is devised by the American child psychologist and pediatrician Arnold Gesell who founded the Clinic of Child Development at Yale in 1911 and directed it for many years.  The Gesell Developmental Schedules are a gauge of the status of a child's motor and language development and personal-social and adaptive behaviors.

For 21 children the age in months at which they first spoke and their Gesell Adaptive Score, which is the result of an aptitude test taken much later was recorded.  These data were originally collected by L. M. Linde of UCLA but were first published by M. R. Mickey, O. J. Dunn, and V. Clark, †Note on the use of stepwise regression in detecting outliers,† Computers and Biomedical Research, 1 (1967), pp. 105†111. 

The data have been used by several authors, e.g.,  N. R. Draper and
J. A. John, †Influential observations and outliers in regression,† 
Technometrics, 23 (1981), pp. 21†26.

Let X be the age in in months a child speaks his/her first word
and let Y be the Gesell adaptive score, a measure of a child†s
aptitude (observed later on). 

Child Age Score        Child Age Score            Child Age Score
----------------------------------------------------------------------------------------------------
1 15 95                       8 11 100                        15 11 102
2 26 71                       9 8 104                          16 10 100
3 10 83                      10 20 94                         17 12 105
4 9 91                        11 7 113                         18 42 57
5 15 102                     12 9 96                          19 17 121
6 20 87                      13 10 83                         20 11 86
7 18 93                      14 11 84                         21 10 100


Correlation of r = ¢0.640 indicates a  moderately strong negative
relationship between age at first word spoken and Gesell score.
 How does the child†s aptitude change with how long it takes them to speak?

Are there any influential observations/potential outliers?

The solution follows a two stage modeling. 
(A) Obtain mean ICPOs. Outside BUGS take reciprocal of mean ICPOs.
(these are CPOs) and consder them as an input for model B.
(B) Read in and calibrate CPOs with posterior ordinates


In [ ]:
# data

## Model A

## Model B
